In [1]:
import random

class Dice:

    def __init__(self):
        self.dice = []

    def roll(self):
        self.reroll(range(0,5))
        return self.dice

    def reroll(self, dice_list):
        self.dice = [ (random.randrange(1, 7) if i in dice_list else self.dice[i]) for i in range(0,5) ]
        return self.dice

    def __str__(self):
        return " ".join([str(i) for i in self.dice])



class Scorecard:
    Numbers = { "primis" : 1, "secundus" : 2, "tertium" : 3, "quartus" : 4, "quintus" : 5, "sextus" : 6 }
    Categories = [ "primis", "secundus", "tertium", "quartus", "quintus", "sextus", "company", "prattle", "squadron", "triplex", "quadrupla", "quintuplicatam", "pandemonium" ]

    def __init__(self):
        self.scorecard = {}
        self.totalscore = 0
        self.bonusscore = 0
        self.bonusflag = False

    def record(self, category, roll):
        dice = roll.dice
        counts = [dice.count(i) for i in range(1,7)]

        if category in self.scorecard:
            print("Error: category already full!")

        if category in Scorecard.Numbers:
            score = counts[Scorecard.Numbers[category]-1] * Scorecard.Numbers[category]
        elif category == "company":
            score = 40 if sorted(dice) == [1,2,3,4,5] or sorted(dice) == [2,3,4,5,6] else 0
        elif category == "prattle":
            score = 30 if (len(set([1,2,3,4]) - set(dice)) == 0 or len(set([2,3,4,5]) - set(dice)) == 0 or len(set([3,4,5,6]) - set(dice)) == 0) else 0
        elif category == "squadron":
            score = 25 if (2 in counts) and (3 in counts) else 0
        elif category == "triplex":
            score = sum(dice) if max(counts) >= 3 else 0
        elif category == "quadrupla":
            score = sum(dice) if max(counts) >= 4 else 0
        elif category == "quintuplicatam":
            score = 50 if max(counts) == 5 else 0
        elif category == "pandemonium":
            score = sum(dice)
        else:
            print("Error: unknown category")
    
        self.scorecard[category] = score
        self.totalscore += score

        # check for bonus
        if not self.bonusflag and len(set(Scorecard.Numbers.keys()) - set(self.scorecard.keys())) == 0:
            self.bonusscore = 35 if sum([ self.scorecard[i] for i in Scorecard.Numbers ]) >= 63 else 0
            self.bonusflag = True
            self.totalscore += self.bonusscore

    def __str__(self):
        s = ""
        for k in Scorecard.Categories:
            s += "      %4d  %-30s \n" % ( (self.scorecard[k] if k in self.scorecard else 0), k)
        s += "      %4s  %-30s \n" % ("" if not self.bonusflag else self.bonusscore, "Bonus")
        s += "      %4d  %-30s \n" % (self.totalscore, "TOTAL")
        return s

In [9]:
from SebastianState import Dice
from SebastianState import Scorecard
import random

def max_score(dice, scorecard):
    counts = [dice.count(i) for i in range(1,7)]
    max_score = ['category', 0]
    total_score = 0

    for category in Scorecard.Categories:
        if category in scorecard.scorecard:
            break
        if category in Scorecard.Numbers:
            score = counts[Scorecard.Numbers[category]-1] * Scorecard.Numbers[category]
            total_score += score
        elif category == "company":
            score = 40 if sorted(dice) == [1,2,3,4,5] or sorted(dice) == [2,3,4,5,6] else 0
            total_score += score
        elif category == "prattle":
            score = 30 if (len(set([1,2,3,4]) - set(dice)) == 0 or len(set([2,3,4,5]) - set(dice)) == 0 or len(set([3,4,5,6]) - set(dice)) == 0) else 0
            total_score += score
        elif category == "squadron":
            score = 25 if (2 in counts) and (3 in counts) else 0
            total_score += score
        elif category == "triplex":
            score = sum(dice) if max(counts) >= 3 else 0
            total_score += score
        elif category == "quadrupla":
            score = sum(dice) if max(counts) >= 4 else 0
            total_score += score
        elif category == "quintuplicatam":
            score = 50 if max(counts) == 5 else 0
            total_score += score
        elif category == "pandemonium":
            score = sum(dice)
            total_score += score
        else:
            print("Error: unknown category")

        if score > max_score[1]:
            max_score = [category, score]
    
    '''if max_score[0] == 'category':
        if 'quintuplicatam' not in scorecard.scorecard:
            max_score[0] = 'quintuplicatam'
        elif 'quadrupla' not in scorecard.scorecard:
            max_score[0] = 'quadrupla'
        else: 
            max_score[0] = random.choice([c for c in Scorecard.Categories if c not in scorecard.scorecard])'''
    
    if max_score[0] == 'category':
        if 'primus' not in scorecard.scorecard:
            max_score[0] = 'primus'
        elif 'secundus' not in scorecard.scorecard:
            max_score[0] = 'secundus'
        if 'tertium' not in scorecard.scorecard:
            max_score[0] = 'tertium'
        elif 'quartus' not in scorecard.scorecard:
            max_score[0] = 'quartus'
        elif 'quintus' not in scorecard.scorecard:
            max_score[0] = 'quintus'
        elif 'sextus' not in scorecard.scorecard:
            max_score[0] = 'sextus'
        else: 
            max_score[0] = random.choice([c for c in Scorecard.Categories if c not in scorecard.scorecard])
            
    return max_score + [total_score]

def exp_value_reroll(dice, strategy, scorecard):
    total_outcomes = 0
    score = 0
    for outcome_a in ([dice[0],] if strategy[0] == False else range(1,7)):
        for outcome_b in ([dice[1],] if strategy[1] == False else range(1,7)):
            for outcome_c in ([dice[2],] if strategy[2] == False else range(1,7)):
                for outcome_d in ([dice[3],] if strategy[3] == False else range(1,7)):
                    for outcome_e in ([dice[4],] if strategy[4] == False else range(1,7)):
                        total_outcomes +=1
                        new_dice = [outcome_a, outcome_b, outcome_c, outcome_d, outcome_e]
                        score += max_score(new_dice, scorecard)[1]
            
    return score * (1/total_outcomes)

def find_best_strategy(dice, scorecard):
    best_score = 0 
    best_strategy = []
    for reroll_a in (True, False):
        for reroll_b in (True, False):
            for reroll_c in (True, False):
                for reroll_d in (True, False):
                    for reroll_e in (True, False):
                        strategy = [reroll_a, reroll_b, reroll_c, reroll_d, reroll_e]
                        score = exp_value_reroll(dice, strategy, scorecard)
                        if score > best_score:
                            best_score = score
                            best_strategy = strategy
    return [i for i, x in enumerate(best_strategy) if x == True]

class SebastianAutoPlayer:

    def __init__(self):
            pass  
    
    def first_roll(self, dice, scorecard):
        dice = dice.dice
        return find_best_strategy(dice, scorecard)

    def second_roll(self, dice, scorecard):
        dice = dice.dice
        return find_best_strategy(dice, scorecard)
      
    def third_roll(self, dice, scorecard):
        dice = dice.dice
        return max_score(dice, scorecard)[0]

In [10]:
# Main program:
#
# Plays 100 games of Sebastian and averages together the scores.
#

scores = []
for i in range(0, 100):
    print("\n\n***** Starting a new game of Sebastian!!")
    d = Dice()
    s = Scorecard()
    ap = SebastianAutoPlayer()

    for i in range(1, 14):
        print("\n * Turn " + str(i))

        # first roll
        d.roll()
        print("   Roll #1: " + str(d))
        which_to_reroll = ap.first_roll(d, s)
        print("Reroll these dice: ", which_to_reroll)

        d.reroll(which_to_reroll)
        print("   Roll #2: " + str(d))
        which_to_reroll = ap.second_roll(d, s)
        print("Reroll these dice: ", which_to_reroll)
        
        d.reroll(which_to_reroll)
        print("   Roll #3: " + str(d))
        category = ap.third_roll(d, s)
        print(category)
        s.record(category, d)
        print(s)

    print("Final score: " + str(s.totalscore))
    scores += [s.totalscore,]

print("Min/max/mean scores: " + str(min(scores)) + " " + str(max(scores)) + " " + str(sum(scores)/100.0))



***** Starting a new game of Sebastian!!

 * Turn 1
   Roll #1: 3 4 6 4 5
Reroll these dice:  [1]
   Roll #2: 3 5 6 4 5
Reroll these dice:  [1]
   Roll #3: 3 6 6 4 5
prattle
         0  primis                         
         0  secundus                       
         0  tertium                        
         0  quartus                        
         0  quintus                        
         0  sextus                         
         0  company                        
        30  prattle                        
         0  squadron                       
         0  triplex                        
         0  quadrupla                      
         0  quintuplicatam                 
         0  pandemonium                    
            Bonus                          
        30  TOTAL                          


 * Turn 2
   Roll #1: 6 2 6 6 6
Reroll these dice:  [1]
   Roll #2: 6 2 6 6 6
Reroll these dice:  [1]
   Roll #3: 6 4 6 6 6
sextus
         0  primis             